In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import string

In [2]:
df_fake=pd.read_csv("C:/Users/Yashit/Downloads/Fake.csv/Fake.csv")
df_true=pd.read_csv("C:/Users/Yashit/Downloads/True.csv/True.csv")

In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
df_fake['class']=0
df_true['class']=1

In [6]:
data=pd.concat((df_fake,df_true),axis=0)

In [7]:
data.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [8]:
data.sample(5)

,title,text,subject,date,class
11953,U.S. Navy chief says forces in Asia may be rei...,YOKOSUKA (Reuters) - The U.S. Navy s top offic...,worldnews,"December 19, 2017",1
10284,Judiciary committee chair prepared to meet cou...,WASHINGTON (Reuters) - Senate Judiciary Commit...,politicsNews,"March 17, 2016",1
20973,[VIDEO] THEY BURNED DOWN CITIES…14 Yr Old Dous...,Barack and Michelle Obama had some unusual ho...,left-news,"Feb 20, 2016",0
3321,Trump aides tell him to keep Sessions as U.S. ...,WASHINGTON (Reuters) - Aides to U.S. President...,politicsNews,"June 8, 2017",1
20375,UK lawmakers back EU withdrawal bill at second...,LONDON (Reuters) - British lawmakers on Tuesda...,worldnews,"September 11, 2017",1


In [9]:
print(data.columns)

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')


In [10]:
data_improved = data.drop(["title","subject","date"],axis=1)

In [11]:
data_improved.sample(5)

,text,class
6163,WASHINGTON (Reuters) - The pomp and circumstan...,1
6976,"Usually, men have all the advantages they ma...",0
21924,Tune in to the Alternate Current Radio Network...,0
14440,Supporting Hillary is about as funny as an ebo...,0
11732,President Donald Trump s Senior Counselor Kell...,0


In [12]:
data_improved.reset_index(inplace=True)

In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\W', ' ', text)
    text = re.sub("https?://\S+|www\.\S+", "", text)
    text = re.sub("<.*?>+", "", text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub("\n","",text)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\W'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\W'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Yashit\AppData\Local\Temp\ipykernel_6752\2983230246.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\Yashit\AppData\Local\Temp\ipykernel_6752\2983230246.py:4: SyntaxWarning: invalid escape sequence '\W'
  text = re.sub('\W', ' ', text)
C:\Users\Yashit\AppData\Local\Temp\ipykernel_6752\2983230246.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub("https?://\S+|www\.\S+", "", text)


In [14]:
data_improved["text"] = data_improved["text"].apply(clean_text)

In [15]:
x = data_improved["text"]
y = data_improved["class"]

x_train, x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25, random_state=42)

In [23]:
vectorizer = TfidfVectorizer()
xv_train=vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)

In [24]:
lr = LogisticRegression()
lr.fit(xv_train, y_train)

LogisticRegression()

In [25]:
pred = lr.predict(xv_test)
lr.score(xv_test,y_test)

0.9858351893095768

In [29]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [31]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr, "lr_model.jb")

['lr_model.jb']

In [37]:
    streamlit run untitled.py

SyntaxError: invalid syntax (4234314677.py, line 1)